In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical


In [47]:
# data=pd.read_csv(r'emoji_data.csv',header=None)
data=pd.read_csv(r'Train.csv',header=None)
data=data.drop(0,axis=1)
data=data.drop(0,axis=0)
data

,1,2
1,Vacation wasted ! #vacation2017 #photobomb #ti...,0
2,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
3,Been friends since 7th grade. Look at us now w...,2
4,This is what it looks like when someone loves ...,3
5,RT @user this white family was invited to a Bl...,3
...,...,...
69996,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69997,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69998,If one of my daughters is wearing this and ask...,2
69999,Guess who whoop people on THEIR homecoming?! #...,3


In [41]:
!pip install emoji
import emoji
print(emoji.emojize(":red_heart:"))

# emoji_dict={
#     0: ":red_heart:",
#     1: ":baseball:",
#     2: ":grinning_face_with_big_eyes:",
#     3: ":disappointed_face:",
#     4: ":fork_and_knife_with_plate:"
# }

df=pd.read_csv('Mapping.csv')
emoji_dict={}
for i in range(len(df.index)):
  key=df['number'][i]
  value=df["emoticons"][i]
  emoji_dict[key]=value
print(emoji_dict)

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])


❤️
{0: '😜', 1: '📸', 2: '😍', 3: '😂', 4: '😉', 5: '🎄', 6: '📷', 7: '🔥', 8: '😘', 9: '❤', 10: '😁', 11: '🇺🇸', 12: '☀', 13: '✨', 14: '💙', 15: '💕', 16: '😎', 17: '😊', 18: '💜', 19: '💯'}


In [48]:
X=data[1].values
y=data[2].values
Y=[]
for item in y:
    Y.append(int(item))
Y=np.asarray(Y)
Y

array([ 0,  1,  2, ...,  2,  3, 14])

Embeddings

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
file=open(r'/content/drive/MyDrive/glove.6B.100d.txt','r',encoding='utf8')
content=file.readlines()
file.close()
content[:4]


['the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062\n',
 ', -0.10767 0.11053 0.59812 -0.54361 0.67396 0.10663 0.038867 0.35481 0.06351 -0.094189 0.15786 -0.81665 0.14172 0.21939 0.58505 -

In [100]:

embeddings={}
for line in content:
    line=line.split()
    embeddings[line[0]]=np.array(line[1:])
# embeddings


In [49]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)
word2index=tokenizer.word_index

xtokens=tokenizer.texts_to_sequences(X)
# xtokens

In [50]:
def max_length(data):
    maxlen=0
    for sent in data:
        maxlen=max(maxlen,len(sent))
    return maxlen

maxlen=max_length(xtokens)
print(maxlen)

36


In [51]:
xtrain=pad_sequences(xtokens,maxlen=maxlen,padding='post',truncating='post')
xtrain

array([[  654,  6433, 16066, ...,     0,     0,     0],
       [  213,  1363,  1191, ...,     0,     0,     0],
       [  136,    94,   510, ...,     0,     0,     0],
       ...,
       [  110,    38,    11, ...,     0,     0,     0],
       [  528,   114, 10004, ...,     0,     0,     0],
       [   26,    15,    10, ...,     0,     0,     0]], dtype=int32)

In [52]:
ytrain=to_categorical(Y)
ytrain

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
embed_size=100
embedding_matrix=np.zeros((len(word2index)+1,embed_size))
print(len(embeddings))

for word,i in word2index.items():
    print(word,i)
    try:
      embed_vector=embeddings[word]
      embedding_matrix[i]=embed_vector
    except:
      continue
embedding_matrix

In [66]:
model=Sequential()
model.add(Embedding(input_dim=len(word2index)+1,output_dim=embed_size,
                    input_length=maxlen,weights=[embedding_matrix],
                    trainable=False))
model.add(LSTM(units=96,return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(20,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [67]:
model.fit(xtrain,ytrain,epochs=100)

Epoch 1/100
2188/2188 [==============================] - 24s 9ms/step - loss: 2.6106 - accuracy: 0.2419
Epoch 2/100
2188/2188 [==============================] - 20s 9ms/step - loss: 2.4618 - accuracy: 0.2845
Epoch 3/100
2188/2188 [==============================] - 20s 9ms/step - loss: 2.3825 - accuracy: 0.3062
Epoch 4/100
2188/2188 [==============================] - 19s 9ms/step - loss: 2.3266 - accuracy: 0.3202
Epoch 5/100
2188/2188 [==============================] - 19s 9ms/step - loss: 2.2780 - accuracy: 0.3350
Epoch 6/100
2188/2188 [==============================] - 19s 8ms/step - loss: 2.2338 - accuracy: 0.3439
Epoch 7/100
2188/2188 [==============================] - 19s 9ms/step - loss: 2.1887 - accuracy: 0.3557
Epoch 8/100
2188/2188 [==============================] - 19s 9ms/step - loss: 2.1454 - accuracy: 0.3663
Epoch 9/100
2188/2188 [==============================] - 20s 9ms/step - loss: 2.1017 - accuracy: 0.3807
Epoch 10/100
2188/2188 [==============================] - 19s 9m

In [68]:
data_test=pd.read_csv(r'Test.csv',header=None)
data_test=data_test.drop(0,axis=1)
data_test=data_test.drop(0,axis=0)
data_test

,1,2
1,0,Thought this was cool...#Repost (get_repost)・・...
2,1,Happy 4th! Corte madera parade. #everytownusa ...
3,2,Luv. Or at least something close to it. @ Unio...
4,3,There's a slice of pie under that whipped crea...
5,5,#thankyou for your thank you We adore you both...
...,...,...
25954,29993,"Rest In Peace, Nana. thanks for playing cards ..."
25955,29994,"I jus wan EAT #chowtime #swt30 @ Hartford, Con..."
25956,29995,Playing Bloon SuperMonkey 2 #bloonsupermonkey2...
25957,29998,Best Snow-cone I've EVER had!!! #bahamabucks #...


In [69]:
X_test=data_test[1].values
y_test=data_test[2].values
Y_test=[]
for item in y_test:
  Y_test.append(int(item))
Y_test=np.asarray(Y_test)
Y_test

ValueError: ignored

In [ ]:
tokenizer_test=Tokenizer()
tokenizer_test.fit_on_texts(X_test)
word2index_test=tokenizer_test.word_index

x_testtokens=tokenizer_test.texts_to_sequences(X_test)

In [ ]:
maxlen_test=max_length(x_testtokens)
print(maxlen_test)

In [ ]:
xtest=pad_sequences(x_testtokens,maxlen=maxlen,padding='post',truncating='post')
ytest=to_categorical(Y_test)

In [ ]:
embedding_matrix=np.zeros((len(word2index)+1,embed_size))

for word,i in word2index_test.items():
    print(word,i)
    try:
      embed_vector=embeddings[word]
      embedding_matrix[i]=embed_vector
    except:
      continue

In [ ]:
from sklearn.metrics import accuracy_score

predictions=model.predict(xtest)
accuracy = accuracy_score(ytest, predictions)

In [99]:
test=['Hello, how are you',"Are you good","lets go to restaurant"]
test_seq=tokenizer.texts_to_sequences(test)
xtest=pad_sequences(test_seq,maxlen=maxlen,padding='post',truncating="post")

y_pred=model.predict(xtest)
y_pred=np.argmax(y_pred,axis=1)

for i in range(len(test)):
    print(test[i],label_to_emoji(y_pred[i]))

1/1 [==============================] - 0s 21ms/step
Yes i'm fine 😍


In [71]:
# for i in range(len(test)):
#     print(test[i],label_to_emoji(y_pred[i]))

Hello, how are you 😍
Are you good ❤
lets go to restaurant 😂
